In [ ]:
%pylab inline

In [ ]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import numpy as np


In [ ]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("Rules")
mydatasetorig = dataiku.Dataset("Shopping_tabular")
dfrules = mydataset.get_dataframe()
dforig = mydatasetorig.get_dataframe()

X = dforig.values
array = np.arange(X.shape[1])
dfrules.sort_values(['antecedents', 'confidence'], ascending=[True, False], inplace=True)

# Map products
dfproducts=dforig*array
mapping = {idx: item for idx, item in enumerate(dforig.columns)}
dfproducts=dfproducts.apply(lambda x: [mapping[i] for i in x])
dfproducts.replace(to_replace=['TICKET'],value='',inplace=True)
products=list(dfproducts.columns.values)
products.remove('TICKET')
del dfproducts['TICKET']


In [ ]:
dfproducts['products']=''

# store list of products purchased
for index, row in dfproducts.iterrows():
    lstfields=[]
    for item in products:
        if len(dfproducts[item][index])>0:
            lstfields.append(item)
    dfproducts['products'][index]=lstfields

# convert antecendents into a list
dfrules['listantec']=''
for index, row in dfrules.iterrows():
    lstantec=[]
    for item in dfrules['antecedents'][index].split(','):
        lstantec.append(item)
    dfrules['listantec'][index]=lstantec
    

In [ ]:
dfproducts['predrule']=''
dfproducts['predconf']=''
dfproducts['predruleid']=''

# search, in antecedents list, for elements contained in products
for index, row in dfproducts.iterrows():
    storerule=[]
    lstproducts=dfproducts['products'][index]
    z = []
    # for each rule check now if each element of listproducts is in antecedents list
    for index2, row2 in dfrules.iterrows():
        count=0
        found=False
        lstantec=dfrules['listantec'][index2]
        conseq=dfrules['consequent'][index2]
        for item in lstproducts:        
            if item in lstantec:
                count+=1
            if item==conseq:
                found=True
        if count==len(lstantec) and not found: # all antecedents from rule have been bought and none is the consequent
            z.append(dfrules['ruleid'][index2]) # rule id
            z.append(dfrules['consequent'][index2])
            z.append(float("{0:.3f}".format(dfrules['confidence'][index2])))
            storerule.append(z)
            z=[]

    # assign now highest confidence
    if len(storerule)>0: #if any rule has been found, store it as prediction:
        conf=0
        highest_conf=0
        icount=-1
        irule=0
        for item3 in storerule:
            icount+=1
            conf=storerule[icount][2]
            if conf>highest_conf:
                highest_conf=conf
                irule=icount

        # store highest confidence rule as prediction
        dfproducts['predrule'][index]=str(storerule[irule][1])
        dfproducts['predconf'][index]=storerule[irule][2]
        dfproducts['predruleid'][index]=storerule[irule][0]

#remove temporary columns
del dfproducts['products']
for item in products:
    del dfproducts[item]

In [ ]:
#add predictions to original file and display it
dfproducts=pd.concat([dforig, dfproducts], axis=1)
dfproducts